**Alumno: Isaias Eleuterio Tenorio Retis**

**Desafio 2**

**Crear sus propios vectores con Gensim basado en lo visto en clases con otro dataset. probar términos de interés y explicar similitudes en el espacio de embedding. Intentar plantear y probar tests de analogías. graficar los embeddings resultantes y sacar conclusiones**

Importamos las librerias necesarias

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import multiprocessing
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

Descargamos y cargamos el dataset

In [2]:
# Cargamos los datos de 20 Newsgroups
newsgroups_data = fetch_20newsgroups(subset='all')
# Convertimos el dataset en un DataFrame para facilitar el procesamiento
df = pd.DataFrame({'text': newsgroups_data.data, 'target': newsgroups_data.target})
df

,text,target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4
...,...,...
18841,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13
18842,From: rdell@cbnewsf.cb.att.com (richard.b.dell...,12
18843,From: westes@netcom.com (Will Estes)\nSubject:...,3
18844,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1


Tokenizamos el texto

In [4]:
# Tokenizamos los textos
sentence_tokens = []
for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row['text']))
# Visualizamos las primeras oraciones tokenizadas
print(sentence_tokens[:2])

[['from', 'mamatha', 'devineni', 'ratnam', 'mr47', 'andrew', 'cmu', 'edu', 'subject', 'pens', 'fans', 'reactions', 'organization', 'post', 'office', 'carnegie', 'mellon', 'pittsburgh', 'pa', 'lines', '12', 'nntp', 'posting', 'host', 'po4', 'andrew', 'cmu', 'edu', 'i', 'am', 'sure', 'some', 'bashers', 'of', 'pens', 'fans', 'are', 'pretty', 'confused', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'posts', 'about', 'the', 'recent', 'pens', 'massacre', 'of', 'the', 'devils', 'actually', 'i', 'am', 'bit', 'puzzled', 'too', 'and', 'a', 'bit', 'relieved', 'however', 'i', 'am', 'going', 'to', 'put', 'an', 'end', 'to', 'non', "pittsburghers'", 'relief', 'with', 'a', 'bit', 'of', 'praise', 'for', 'the', 'pens', 'man', 'they', 'are', 'killing', 'those', 'devils', 'worse', 'than', 'i', 'thought', 'jagr', 'just', 'showed', 'you', 'why', 'he', 'is', 'much', 'better', 'than', 'his', 'regular', 'season', 'stats', 'he', 'is', 'also', 'a', 'lot', 'fo', 'fun', 'to', 'watch', 'in', 'the', 'playoffs'

Entrenamos el modelo Word2Vec

In [7]:
# Averiguo el número de núcleos disponibles en mi máquina para asignarlo en workers
print(multiprocessing.cpu_count())

2


In [8]:
# Creamos y entrenamos el modelo Word2Vec (Skipgram)
w2v_model = Word2Vec(
    sentences=sentence_tokens,
    vector_size=300,
    window=5,
    min_count=5,
    workers=2,  # Es el valor obtenido en el comando anterior
    sg=1,  # Skip-gram
    epochs=20,
    compute_loss=True
)
# Guardamos el modelo
w2v_model.save("word2vec_20newsgroups.model")

Probamos términos de interes

In [9]:
# Palabras más similares a 'computer'
print(w2v_model.wv.most_similar(positive=["computer"], topn=10))

# Palabras más similares a 'love'
print(w2v_model.wv.most_similar(positive=["love"], topn=10))

# Probamos con una palabra que no esté en el vocabulario definido
try:
    print(w2v_model.wv.most_similar(positive=["nonexistentword"], topn=10))
except KeyError as e:
    print(e)

[('science', 0.5469357371330261), ('umea', 0.43562185764312744), ('itti', 0.41718971729278564), ('rootstown', 0.4047037661075592), ('523296', 0.4039481580257416), ('vpi', 0.4024342894554138), ('u23590', 0.39865538477897644), ('engineering', 0.3978457748889923), ('jstmp', 0.39059269428253174), ('totowa', 0.3895168602466583)]
[('undying', 0.44323551654815674), ('salesmen', 0.43855565786361694), ("'56", 0.4379690885543823), ('learner', 0.4377578794956207), ('scots', 0.4301436245441437), ('yankovic', 0.4263051152229309), ('diety', 0.42525503039360046), ('gaelic', 0.4226386845111847), ('motherly', 0.4172956049442291), ('metaphors', 0.4166799485683441)]
"Key 'nonexistentword' not present in vocabulary"


**Comentario**

El modelo Word2Vec encuentra palabras similares a computer y love en el espacio de embedding. las palabras relacionadas con tecnología e informática están vinculadas a computer , mientras que las palabras relacionadas con sentimiento y emoción están vinculadas a love.

Por último,El modelo no puede encontrar palabras similares a una palabra desconocida

Pruebas de analogías

In [10]:
# Probamos una analogía: 'king' - 'man' + 'woman' = ?
print(w2v_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))
# Otra analogía: 'paris' - 'france' + 'germany' = ?
print(w2v_model.wv.most_similar(positive=['paris', 'germany'], negative=['france'], topn=1))

[('sysops', 0.3362012505531311)]
[('braunschweig', 0.37718236446380615)]


**Comentario**

Se realizaron pruebas de analogías con el modelo Word2Vec. la analogía 'king' - 'man' + 'woman' produjo "sysops" con una similitud de 0.3362.
la analogía 'paris' - 'france' + 'germany' produjo "braunschweig" con una similitud de 0.3771.

Los resultados muestran la capacidad del modelo para realizar analogías complejas.

Reducimos dimensionalidad y probamos embeddings

In [11]:
def reduce_dimensions(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    if num_dimensions == 2:
        tsne = TSNE(n_components=2, random_state=0)
        vectors = tsne.fit_transform(vectors)
    elif num_dimensions == 3:
        tsne = TSNE(n_components=3, random_state=0)
        vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [12]:
# Reducimos las dimensiones a 2D
vecs, labels = reduce_dimensions(w2v_model)

# Definimos el tamaño del texto y los puntos y visualizamos el gráfico
MAX_WORDS = 200
fig = px.scatter(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], text=labels[:MAX_WORDS])

# Agregamos colores y ajuste de tamaño de los puntos y texto
fig.update_traces(marker=dict(size=8, color="blue"), textposition="top center")
fig.update_layout(title="Word2Vec Embeddings for 20 Newsgroups Dataset",
                  xaxis_title="Dimension 1", yaxis_title="Dimension 2",
                  font=dict(size=12))
fig.show()

**Conclusión**

El gráfico de embedding muestra agrupamientos semánticos claros, donde palabras relacionadas aparecen juntas, como por ejemplo "people", "Who", "about" en el centro, indicando que el modelo Word2Vec ha capturado bien las relaciones contextuales.

Por otro lado, las palabras técnicas como "computer" y "windows" forman cluster específicos. Algunas palabras dispersas como "god" o "tin", podrian ser menos frecuentes o quizas tener significados mas específicos.

En general, el modelo ha capturado adecuadamente las relaciones semánticas del corpus.